In [ ]:
import shap
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from raiwidgets import ModelAnalysisDashboard
from responsibleai import ModelAnalysis

### Use adult census dataset

In [ ]:
X, y = shap.datasets.adult()
target_feature = 'income'
categorical_features = ['Workclass', 'Education-Num', 'Marital Status',
                        'Occupation', 'Relationship', 'Race', 'Sex', 'Country']

y = [1 if x else 0 for x in y]

n_samples = 10000
X, y = sklearn.utils.resample(X, y, n_samples=n_samples, random_state=7, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test

### Train a model

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

### Compute Responsible AI model and dataset insights

In [ ]:
model_analysis = ModelAnalysis(model, train_data, test_data, target_feature, 'classification',
                               categorical_features=categorical_features)

# Queue Responsible AI insights
model_analysis.explainer.add()
model_analysis.counterfactual.add(10, desired_class='opposite')
model_analysis.error_analysis.add()
model_analysis.causal.add(treatment_features=['Relationship'])

# Compute insights
model_analysis.compute()

### Visualize insights in the model analysis dashboard

In [ ]:
ModelAnalysisDashboard(model_analysis)